## Розширений план практичного заняття  
**Тема 4. Заняття 13. Практичне використання репозиторіїв аналітичних моделей в межах виконання індивідуальних (групових) проектів**

---

### **1. Мета заняття**
- Забезпечити слухачам практичні навички роботи з репозиторіями попередньо навчених моделей (Transfer Learning) та методами Reinforcement Learning (RL).
- Навчити слухачів ефективно використовувати попередньо навчені моделі для вирішення конкретних інформаційно-аналітичних завдань у військовій сфері.
- Сприяти розвитку командної роботи та співпраці через виконання групових проектів.

### **2. Очікувані результати**
Після завершення заняття слухачі зможуть:
- Використовувати репозиторії аналітичних моделей для вирішення індивідуальних завдань.
- Співпрацювати в групах для розробки комплексних рішень, використовуючи попередньо навчені моделі та методи RL.
- Презентувати та обговорювати результати своїх проектів, отримуючи зворотний зв'язок для подальшого вдосконалення.

### **3. Структура заняття**

| Час        | Активність                                                                                   |
|------------|----------------------------------------------------------------------------------------------|
| 0-10 хв    | **Вступ та огляд**<br>- Привітання.<br>- Огляд цілей та структури заняття.                 |
| 10-30 хв   | **Індивідуальна робота: Використання репозиторіїв аналітичних моделей**<br>- Інструктаж та пояснення завдань.<br>- Демонстрація прикладів використання моделей з TensorFlow Hub/PyTorch Model Zoo. |
| 30-60 хв   | **Практична частина: Fine-Tuning попередньо навчених моделей**<br>- Завдання для слухачів: вибрати модель та адаптувати її до конкретного завдання (наприклад, класифікація зображень військової техніки).<br>- Надання кодових прикладів та підтримка викладача. |
| 60-70 хв   | **Перерва**                                                                                   |
| 70-90 хв   | **Групова робота: Використання RL у військових сценаріях**<br>- Формування груп.<br>- Визначення завдань для груп: розробка RL-агента для симуляції бойової ситуації або оптимізації логістичних маршрутів. |
| 90-120 хв  | **Практична частина: Розробка RL-агентів**<br>- Надання прикладів коду (DQN, Actor-Critic).<br>- Групове програмування та інтеграція моделей у симульоване середовище. |
| 120-130 хв | **Презентація групових проектів**<br>- Кожна група презентує свою концепцію та результати.<br>- Обговорення та зворотний зв'язок від викладача та інших груп. |
| 130-140 хв | **Обговорення та аналіз**<br>- Висвітлення успіхів та труднощів.<br>- Рекомендації щодо покращення проектів. |
| 140-150 хв | **Підсумки та завершення**<br>- Основні висновки заняття.<br>- Оголошення домашнього завдання (за потреби). |

---

### **4. Детальний навчальний контент**

#### **4.1. Індивідуальна робота: Використання репозиторіїв аналітичних моделей**

##### **4.1.1. Інструктаж та пояснення завдань**
- **Завдання:** Кожен слухач обирає одну з попередньо навчених моделей з TensorFlow Hub або PyTorch Model Zoo для вирішення конкретної задачі (наприклад, класифікація зображень військової техніки, аналіз текстових донесень).
- **Мета:** Навчитися адаптувати попередньо навчену модель до власного набору даних через fine-tuning.

##### **4.1.2. Демонстрація прикладів використання моделей**

**Приклад: Fine-Tuning ResNet50 для класифікації військової техніки (PyTorch)**

```python
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
from torch.optim import lr_scheduler
import time
import copy
import os

# 1. Підготовка даних
data_dir = 'path_to_your_dataset'  # Вкажіть шлях до вашого набору даних
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), 
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                              batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 2. Завантаження попередньо навченого ResNet
model = models.resnet50(pretrained=True)

# 3. Заморожування шарів
for param in model.parameters():
    param.requires_grad = False

# 4. Заміна вихідного шару
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

# 5. Визначення критерію та оптимізатора
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# 6. Навчання моделі
num_epochs = 25
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
since = time.time()

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)
    
    # Кожен епох розділений на тренувальний та валідаційний
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # режим навчання
        else:
            model.eval()   # режим валідації
        
        running_loss = 0.0
        running_corrects = 0
        
        # Ітерація по даних
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            # Прямий прохід
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                # Зворотний прохід + оптимізація тільки в тренувальному режимі
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Збереження найкращої моделі
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
    
    print()
    
time_elapsed = time.time() - since
print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
print(f'Best val Acc: {best_acc:4f}')

# Завантаження найкращої моделі
model.load_state_dict(best_model_wts)

# Збереження моделі
torch.save(model.state_dict(), 'fine_tuned_resnet.pth')
```

**Пояснення коду:**
1. **Підготовка даних:** Використовуються трансформації для попередньої обробки зображень (Resize, RandomHorizontalFlip, ToTensor, Normalize).
2. **Завантаження моделі:** Використовується попередньо навчена модель ResNet50 з бібліотеки torchvision.
3. **Заморожування шарів:** Всі шари попередньо навченої моделі заморожуються (`requires_grad = False`), щоб зберегти вже набуті представлення.
4. **Заміна вихідного шару:** Вихідний шар змінюється на новий, що відповідає кількості класів у вашому наборі даних.
5. **Визначення критерію та оптимізатора:** Використовується CrossEntropyLoss та Adam-оптимізатор для навчання нового шару.
6. **Навчання:** Модель навчається лише останнього шару на новому наборі даних. Під час тренувальної фази відбувається оновлення ваг, а під час валідаційної фази модель оцінюється без оновлення ваг.
7. **Збереження моделі:** Збереження найкращої моделі на основі валідаційної точності.

##### **4.1.3. Завдання для слухачів**

**Завдання:**
1. **Вибір моделі:** Оберіть попередньо навчену модель з TensorFlow Hub або PyTorch Model Zoo, яка найбільше підходить для вашої задачі.
2. **Адаптація моделі:** Виконайте fine-tuning моделі на вашому наборі даних.
3. **Оцінка результатів:** Перевірте точність моделі на валідаційних даних та оптимізуйте параметри при необхідності.
4. **Презентація результатів:** Підготуйте коротку презентацію з описом вашої задачі, вибору моделі, процесу адаптації та отриманих результатів.

---

#### **4.2. Групова робота: Використання RL у військових сценаріях**

##### **4.2.1. Інструктаж та пояснення завдань**

**Завдання:** Кожна група обирає військову інформаційно-аналітичну задачу, для якої застосовуватиме методи Reinforcement Learning. Наприклад, розробка RL-агента для оптимізації логістичних маршрутів або управління безпілотними літальними апаратами (БПЛА) у симульованих бойових умовах.

**Мета:** Розробити концепцію та реалізувати простий RL-агент, який може вирішувати обрану задачу в симульованому середовищі.

##### **4.2.2. Приклад практичної частини: Створення RL-агента для оптимізації логістичних маршрутів**

**Задача:** Оптимізація маршруту доставки ресурсів у змінних умовах бойової обстановки.

**Інструменти:** Python, PyTorch, OpenAI Gym (створення власного середовища або використання існуючих середовищ).

**Приклад коду: Реалізація простого RL-агента за допомогою DQN**

```python
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque

# 1. Визначення нейронної мережі
class DQN(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=24):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, action_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.out(x)

# 2. Ініціалізація середовища
env = gym.make('CartPole-v1')  # Для групового проекту рекомендується створити власне середовище
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# 3. Параметри DQN
batch_size = 64
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001
memory = deque(maxlen=2000)

# 4. Ініціалізація моделі та оптимізатора
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DQN(state_size, action_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# 5. Функція для вибору дії
def act(state):
    global epsilon
    if random.random() < epsilon:
        return random.randrange(action_size)
    state = torch.FloatTensor(state).unsqueeze(0).to(device)
    with torch.no_grad():
        q_values = model(state)
    return torch.argmax(q_values).item()

# 6. Функція для навчання моделі
def replay():
    global epsilon
    if len(memory) < batch_size:
        return
    minibatch = random.sample(memory, batch_size)
    states, actions, rewards, next_states, dones = zip(*minibatch)
    
    states = torch.FloatTensor(states).to(device)
    actions = torch.LongTensor(actions).unsqueeze(1).to(device)
    rewards = torch.FloatTensor(rewards).unsqueeze(1).to(device)
    next_states = torch.FloatTensor(next_states).to(device)
    dones = torch.FloatTensor(dones).unsqueeze(1).to(device)
    
    # Поточні Q-значення
    q_values = model(states).gather(1, actions)
    
    # Мети Q-значень
    next_q_values = model(next_states).max(1)[0].unsqueeze(1)
    targets = rewards + (gamma * next_q_values * (1 - dones))
    
    # Обчислення втрат та зворотний прохід
    loss = criterion(q_values, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Зменшення epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

# 7. Навчання агента
num_episodes = 1000
for e in range(num_episodes):
    state = env.reset()
    done = False
    total_reward = 0
    while not done:
        action = act(state)
        next_state, reward, done, _ = env.step(action)
        memory.append((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward
        replay()
    print(f'Episode {e+1}/{num_episodes}, Reward: {total_reward}, Epsilon: {epsilon:.2f}')

# 8. Збереження моделі
torch.save(model.state_dict(), 'dqn_cartpole.pth')
```

**Пояснення коду:**
1. **Нейронна мережа DQN:** Простий багатошаровий перцептрон для оцінки Q-функції.
2. **Середовище:** Використовується OpenAI Gym середовище CartPole для демонстрації. У групових проектах рекомендується створити власне середовище, що відображає військовий сценарій.
3. **Параметри DQN:** Визначаються основні параметри навчання, такі як розмір батчу, коефіцієнт дисконтування, параметри epsilon.
4. **Функція `act`:** Вибір дії на основі політики epsilon-greedy.
5. **Функція `replay`:** Навчання моделі на випадкових батчах з пам'яті.
6. **Навчання агента:** Проходить через визначену кількість епох, де агент взаємодіє зі середовищем та навчається.
7. **Збереження моделі:** Збереження навчених ваг для подальшого використання.

##### **4.2.3. Завдання для груп**

**Завдання:**
1. **Вибір сценарію:** Оберіть військову інформаційно-аналітичну задачу, для якої застосовуватиметься Reinforcement Learning. Наприклад:
   - Оптимізація логістичних маршрутів доставки ресурсів.
   - Управління безпілотними літальними апаратами (БПЛА) у бойових умовах.
   - Автоматизоване планування та виконання бойових операцій у змінних сценаріях.
2. **Розробка середовища:** Створіть або адаптуйте симульоване середовище, яке відображає обрану задачу.
3. **Розробка агента:** Виберіть алгоритм RL (наприклад, DQN, Actor-Critic) та реалізуйте агента, здатного вирішувати обрану задачу.
4. **Навчання та тестування:** Навчіть агента у створеному середовищі та оцініть його продуктивність.
5. **Презентація:** Підготуйте коротку презентацію з описом завдання, розробленого рішення та отриманих результатів.

##### **4.2.4. Практична частина: Розробка RL-агента**

**Приклад: Використання DQN для оптимізації логістичних маршрутів**

**Сценарій:** Агент повинен навчитися вибирати оптимальний маршрут доставки ресурсів між двома точками на карті з урахуванням змінних умов (наприклад, наявність загроз, обмеження на час).

**Кроки:**
1. **Створення середовища:** Використання бібліотеки OpenAI Gym для створення власного середовища або модифікація існуючого.
2. **Розробка та навчання агента:** Використання методу DQN для навчання агента вибору маршруту.
3. **Оцінка:** Тестування агента на різних сценаріях та аналіз його продуктивності.

**Приклад коду: Створення власного середовища**

```python
import gym
from gym import spaces
import numpy as np

class LogisticsEnv(gym.Env):
    """
    Просте середовище для оптимізації логістичних маршрутів.
    """
    def __init__(self):
        super(LogisticsEnv, self).__init__()
        # Простір дій: переміщення в 4 напрямках
        self.action_space = spaces.Discrete(4)
        # Простір станів: координати (x, y) на карті 10x10
        self.observation_space = spaces.Box(low=0, high=9, shape=(2,), dtype=np.int32)
        # Початкова позиція та ціль
        self.start = np.array([0, 0])
        self.goal = np.array([9, 9])
        self.state = self.start.copy()
        self.done = False

    def reset(self):
        self.state = self.start.copy()
        self.done = False
        return self.state

    def step(self, action):
        if self.done:
            return self.state, 0, self.done, {}
        
        # Визначення нової позиції
        if action == 0:   # Вгору
            self.state[1] = min(self.state[1] + 1, 9)
        elif action == 1: # Вниз
            self.state[1] = max(self.state[1] - 1, 0)
        elif action == 2: # Вліво
            self.state[0] = max(self.state[0] - 1, 0)
        elif action == 3: # Вправо
            self.state[0] = min(self.state[0] + 1, 9)
        
        # Перевірка досягнення цілі
        if np.array_equal(self.state, self.goal):
            reward = 10
            self.done = True
        else:
            reward = -1  # Негативна винагорода за кожен крок
        
        return self.state, reward, self.done, {}

    def render(self, mode='human'):
        grid = np.zeros((10, 10), dtype=str)
        grid[:] = '.'
        grid[self.goal[1], self.goal[0]] = 'G'
        grid[self.state[1], self.state[0]] = 'A'
        print('\n'.join([' '.join(row) for row in grid]))
```

**Приклад коду: Навчання агента у створеному середовищі**

```python
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque

# 1. Визначення нейронної мережі
class DQN(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=24):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, action_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.out(x)

# 2. Ініціалізація середовища
env = LogisticsEnv()
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# 3. Параметри DQN
batch_size = 32
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001
memory = deque(maxlen=5000)

# 4. Ініціалізація моделі та оптимізатора
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DQN(state_size, action_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# 5. Функція для вибору дії
def act(state):
    global epsilon
    if random.random() < epsilon:
        return random.randrange(action_size)
    state = torch.FloatTensor(state).unsqueeze(0).to(device)
    with torch.no_grad():
        q_values = model(state)
    return torch.argmax(q_values).item()

# 6. Функція для навчання моделі
def replay():
    global epsilon
    if len(memory) < batch_size:
        return
    minibatch = random.sample(memory, batch_size)
    states, actions, rewards, next_states, dones = zip(*minibatch)
    
    states = torch.FloatTensor(states).to(device)
    actions = torch.LongTensor(actions).unsqueeze(1).to(device)
    rewards = torch.FloatTensor(rewards).unsqueeze(1).to(device)
    next_states = torch.FloatTensor(next_states).to(device)
    dones = torch.FloatTensor(dones).unsqueeze(1).to(device)
    
    # Поточні Q-значення
    q_values = model(states).gather(1, actions)
    
    # Мети Q-значень
    next_q_values = model(next_states).max(1)[0].unsqueeze(1)
    targets = rewards + (gamma * next_q_values * (1 - dones))
    
    # Обчислення втрат та зворотний прохід
    loss = criterion(q_values, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Зменшення epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

# 7. Навчання агента
num_episodes = 500
for e in range(num_episodes):
    state = env.reset()
    done = False
    total_reward = 0
    while not done:
        action = act(state)
        next_state, reward, done, _ = env.step(action)
        memory.append((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward
        replay()
    print(f'Episode {e+1}/{num_episodes}, Reward: {total_reward}, Epsilon: {epsilon:.2f}')

# 8. Збереження моделі
torch.save(model.state_dict(), 'dqn_logistics.pth')
```

**Пояснення коду:**
1. **Створення середовища:** Використовується власне середовище `LogisticsEnv`, яке симулює задачу оптимізації маршрутів.
2. **Нейронна мережа DQN:** Модель оцінює Q-функцію для вибору дій агента.
3. **Параметри DQN:** Визначаються основні параметри навчання, включаючи розмір батчу, коефіцієнт дисконтування та параметри epsilon.
4. **Функція `act`:** Вибір дії за політикою epsilon-greedy.
5. **Функція `replay`:** Навчання моделі на випадкових батчах з пам'яті.
6. **Навчання агента:** Агент навчається проходити через серії епізодів у середовищі, оптимізуючи свою стратегію.
7. **Збереження моделі:** Збереження навчених ваг для подальшого використання або аналізу.

---

### **5. Груповий проєкт**

#### **5.1. Завдання для груп**

**Група 1:** **Transfer Learning для класифікації бойової техніки**
- **Задача:** Розробити модель для класифікації типів військової техніки на зображеннях супутникових знімків.
- **Кроки:**
  1. Вибір попередньо навченої моделі (наприклад, ResNet50).
  2. Збір та анотування набору даних зображень різних типів техніки.
  3. Адаптація моделі через fine-tuning.
  4. Оцінка та тестування моделі.
  5. Презентація результатів.

**Група 2:** **Reinforcement Learning для управління БПЛА**
- **Задача:** Створити RL-агента для оптимізації маршрутів безпілотного літального апарата (БПЛА) у бойових умовах.
- **Кроки:**
  1. Визначення середовища симуляції (наприклад, використання OpenAI Gym або створення власного середовища).
  2. Розробка та навчання RL-агента (DQN, Actor-Critic).
  3. Тестування агента у різних сценаріях.
  4. Аналіз продуктивності та оптимізація.
  5. Презентація результатів.

#### **5.2. Вимоги до проєкту**
- **Докладний опис:** Чітко визначена задача, вибір методів та інструментів, опис процесу розробки та навчання моделі.
- **Використання коду:** Інтеграція прикладів коду або розробка власних реалізацій, адаптованих до задачі.
- **Презентація:** Коротка (5-10 хв) презентація перед класом з демонстрацією результатів та обговоренням викликів.
- **Документація:** Підготовка документації з описом кроків розробки, використаних технологій та отриманих результатів.

---

### **6. Підсумки та обговорення**

- **Основні висновки:**
  - **Transfer Learning** дозволяє ефективно використовувати попередньо навчені моделі для нових задач, що значно знижує час та ресурси на навчання.
  - **Reinforcement Learning** надає можливість створювати адаптивних агентів, здатних приймати оптимальні рішення в складних та змінних середовищах.
  - **Практичні завдання** демонструють реальну застосовність методів Transfer Learning та RL у військових інформаційно-аналітичних системах.
  
- **Обговорення питань:**
  - Які були основні труднощі при адаптації моделей?
  - Як можна покращити точність моделей?
  - Які додаткові методи або технології можна інтегрувати для покращення інформаційно-аналітичного забезпечення?
  
- **Домашнє завдання (за потреби):**
  - Інтегрувати навчену модель Transfer Learning у симульовану інформаційну систему.
  - Розробити та навчити більш складного RL-агента для обраної задачі, враховуючи додаткові параметри або умови.

---

### **7. Рекомендована література та джерела**

1. **Ian Goodfellow, Yoshua Bengio, Aaron Courville.** *Deep Learning*. MIT Press, 2016.
2. **Francois Chollet.** *Deep Learning with Python*. Manning Publications, 2018.
3. **Sutton R.S., Barto A.G.** *Reinforcement Learning: An Introduction*. MIT Press, 2018.
4. **Jeremy Howard, Sylvain Gugger.** *Deep Learning for Coders with Fastai and PyTorch*. O'Reilly Media, 2020.
5. **TensorFlow Hub:** [https://tfhub.dev/](https://tfhub.dev/)
6. **PyTorch Model Zoo:** [https://pytorch.org/docs/stable/torchvision/models.html](https://pytorch.org/docs/stable/torchvision/models.html)
7. **Hugging Face Model Hub:** [https://huggingface.co/models](https://huggingface.co/models)
8. **OpenAI Gym:** [https://gym.openai.com/](https://gym.openai.com/)
9. **Документація PyTorch:** [https://pytorch.org/docs/stable/index.html](https://pytorch.org/docs/stable/index.html)
10. **Документація TensorFlow:** [https://www.tensorflow.org/learn](https://www.tensorflow.org/learn)
11. **Матеріали конференцій MILCOM, ICLR, ICML, NeurIPS:** сучасні дослідження з машинного та глибокого навчання, зокрема у військовому та безпековому контексті.

---

### **8. Додаткові ресурси та матеріали**

- **Відео-лекції та туторіали:**
  - YouTube-канали, такі як **deeplizard**, **sentdex**, **PyTorch**, **TensorFlow**, які пропонують детальні відео-уроки з Transfer Learning та Reinforcement Learning.
- **Онлайн-курси:**
  - **Coursera:** *Deep Learning Specialization* від Andrew Ng.
  - **edX:** *Reinforcement Learning* від Microsoft.
  - **Udemy:** Курси з Transfer Learning та RL.
- **Документація та туторіали:**
  - **PyTorch Tutorials:** [https://pytorch.org/tutorials/](https://pytorch.org/tutorials/)
  - **TensorFlow Tutorials:** [https://www.tensorflow.org/tutorials](https://www.tensorflow.org/tutorials)
  - **Hugging Face Tutorials:** [https://huggingface.co/transformers/tutorials.html](https://huggingface.co/transformers/tutorials.html)
  - **OpenAI Gym Documentation:** [https://gym.openai.com/docs/](https://gym.openai.com/docs/)

---

**Примітка:** Для ефективного засвоєння матеріалу рекомендується активна участь у практичних завданнях, обговореннях та групових проєктах. Викладач може адаптувати складність завдань залежно від рівня підготовки слухачів та доступних ресурсів.

---

**Завершення:** Це практичне заняття спрямоване на закріплення теоретичних знань з Transfer Learning та Reinforcement Learning через практичну роботу з попередньо навченими моделями та розробку RL-агентів. Слухачі отримають цінний досвід у використанні сучасних методів машинного навчання для вирішення конкретних військових інформаційно-аналітичних задач, що сприятиме їх професійному розвитку та підвищенню компетентності у сфері забезпечення національної безпеки.